In this code, we create an automatic chart generator for each country.  The charts we generate are the average temperature anomaly plots, the heatmaps, and the Ed Hawkins climate stripes.

In [9]:
%matplotlib inline

from IPython.display import clear_output
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import Wrangler as Wr
import Countrypack as Cp

# Function to expand confidence intervals for temperature data
def makeConfidenceInt(data,period):
    upper = period + 'Up'
    lower = period + 'Lo'
    anomaly= period + 'Anomaly'
    uncertainty = period + 'Unc'
    data[upper] = data[anomaly] + data[uncertainty]
    data[lower] = data[anomaly] - data[uncertainty]
    

In [10]:
# Load Country Temperature data
filePath = Wr.makeDirFile('Temp_countries')
cTempdata = pd.read_csv(filePath)

# Convert date to datetime
cTempdata.Date = pd.to_datetime(cTempdata.Date, format='%Y-%m-%d', errors='coerce')

# Create a Month name column
cTempdata['strMonth'] = cTempdata.Date.dt.month_name()

# Create non-index Date
cTempdata['nonIndexDate'] = cTempdata.Date

# set date index
cTempdata = cTempdata.set_index('Date')

# Convert string month to categorical datatype
months_order = ['January', 'February', 'March', 'April','May','June',
                'July','August','September','October','November','December']
cTempdata.strMonth = pd.Categorical(cTempdata.strMonth, 
                     categories=months_order, ordered=True)

# Generate Confidence intervals
for period in ['Monthly','Annual','TenYear','TwentyYear']:
    makeConfidenceInt(cTempdata,period)
    
# Remove some of the dupicated countries that end with '(Europe)'
cTempdata = cTempdata[~cTempdata.Country.str.contains('Europe')]

# Replace Congo (Democratice Republic of the) with DRC
cTempdata['Country'] = cTempdata['Country'].replace('Congo (Democratic Republic of the)','DRC')

In [13]:
# List the country names that we can visualise
cTempdata.Country.unique()

array(['Afghanistan', 'Åland', 'Albania', 'Algeria', 'American Samoa',
       'Andorra', 'Angola', 'Anguilla', 'Antarctica',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Baker Island', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium',
       'Belize', 'Benin', 'Bhutan', 'Bolivia',
       'Bonaire, Saint Eustatius and Saba', 'Bosnia and Herzegovina',
       'Botswana', 'Brazil', 'British Virgin Islands', 'Bulgaria',
       'Burkina Faso', 'Burma', 'Burundi', 'Cambodia', 'Cameroon',
       'Canada', 'Cape Verde', 'Cayman Islands',
       'Central African Republic', 'Chad', 'Chile', 'China',
       'Christmas Island', 'Colombia', 'Comoros', 'Congo', 'DRC',
       'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba', 'Curaçao',
       'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominica',
       'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador',
       'Equatorial Guinea', 'Eritrea', 'Eston

In [15]:
# Run indefinite loop
while True:
    
    # Prompt for country entry
    print("Type 'done' when you are finished.")
    response = input('Country name: ')
    sCountry = str(response)
    
    # Quit if user enters done
    if response.lower() == 'done':
        # Clear all outputs
        clear_output()
        break

    else:
        # Clear outputs and reset
        clear_output()
        
        # Show temperature lines
        Cp.tempLines(cTempdata,sCountry)
        try:
            # Show heatmap
            Cp.heatmapper(cTempdata,sCountry,'MonthlyAnomaly')
        except:
            print('No data at monthly resolution, trying annual...')
            try:
                Cp.heatmapper(cTempdata,sCountry,'AnnualAnomaly')
            except:
                print('No data either.')

        # Show climate stripes
        sns.reset_defaults()
        Cp.countryStripes(cTempdata,sCountry,xtrigger=True)